In [1]:
import os
os.environ["LANGFUSE_PUBLIC_KEY"] = "pk-lf-8c74f297-f026-484d-93d3-c048f8e2b386"
os.environ["LANGFUSE_SECRET_KEY"] = "sk-lf-f5823dc5-753d-4db7-a052-0bf0cfe9c3cf"
os.environ["LANGFUSE_HOST"] = "https://cloud.langfuse.com"

In [2]:
from anthropic import AnthropicBedrock
from application import Application
from compiler.core import Compiler
import tempfile

In [3]:
compiler = Compiler("botbuild/tsp_compiler", "botbuild/app_schema")
client = AnthropicBedrock(aws_profile="dev", aws_region="us-west-2")

In [4]:
tempdir = tempfile.TemporaryDirectory()
application = Application(client, compiler, "templates", tempdir.name)

In [5]:
application_description = """
Bot that helps me planning my YouTube content and posting schedule.
""".strip()

In [6]:
application_description = "Bot that greets user by name and returns personalized greeting."

In [ ]:
my_bot = application.create_bot(application_description, bot_id="handler_tests")

Compiling TypeSpec...
Compiling Typescript Schema Definitions...
Compiling Drizzle...
Generating Router...
Compiling Handler Tests...
Compiling Handlers...
Generating Application...


In [7]:
my_bot.__dict__

{'typespec': TypespecOut(reasoning='I expect user to send messages like "Plan a video about TypeScript" or "Schedule a video for next week".\nLLM can extract and infer the arguments from plain text and pass them to the handler:\n"Plan a video about TypeScript" -> planContent({topic: "TypeScript", description: "..."})\n"Schedule a video for next week" -> scheduleContent({contentId: "...", publishDate: "2024-01-20"})\n"List my planned videos" -> listContent({status: "planned"})\n\nKey functionalities needed:\n- Plan new content with topic and description\n- Schedule content for publication\n- List planned/scheduled content\n- Update content details', typespec_definitions='model Content {\n    contentId: string;\n    topic: string;\n    description: string;\n    status: string;\n    duration: duration;\n    tags: string[];\n}\n\nmodel ContentSchedule {\n    contentId: string;\n    publishDate: plainDate;\n    publishTime: plainTime;\n}\n\nmodel ContentFilter {\n    status: string;\n    fr

In [8]:
print(my_bot.typescript_schema.typescript_schema)

import { z } from 'zod';

// Person schema
export const personSchema = z.object({
    name: z.string(),
    preferredLanguage: z.string(),
});

export type Person = z.infer<typeof personSchema>;

// GreetingPreference schema
export const greetingPreferenceSchema = z.object({
    formal: z.boolean(),
    timeOfDay: z.boolean(),
});

export type GreetingPreference = z.infer<typeof greetingPreferenceSchema>;

// GreetingRequest schema
export const greetingRequestSchema = z.object({
    person: personSchema,
    preferences: greetingPreferenceSchema,
});

export type GreetingRequest = z.infer<typeof greetingRequestSchema>;

// Function declarations for the GreetingBot interface
export declare function setGreetingPreference(options: GreetingPreference): Promise<void>;
export declare function greet(options: GreetingRequest): Promise<string>;


In [9]:
application.generation_dir

'/tmp/tmp5hgtatoo/generated/generation-1'

In [8]:
import shutil
shutil.copytree(f"{application.generation_dir}/app_schema", "../../bot_results/bot_zod_gen")

'../../bot_results/bot_zod_gen'

In [9]:
my_bot.router.functions

[{'name': 'createVideoPlan',
  'description': 'Creates a detailed plan for a new video content including title, description, estimated duration, target audience, tags, and notes',
  'examples': ['Help me plan a video about gardening tips',
   'I want to create a video about cooking basics',
   'Create a content plan for my tech review video']},
 {'name': 'scheduleVideo',
  'description': 'Schedules a video for publication at a specific date and time',
  'examples': ['Schedule my gardening video for next Tuesday at 3pm',
   'I want to publish my cooking video on December 1st',
   'Set my tech review to go live tomorrow morning at 9am']},
 {'name': 'getScheduledVideos',
  'description': 'Retrieves a list of scheduled videos within a specified date range',
  'examples': ['Show me all videos scheduled for next week',
   'What videos are planned between March 1st and March 15th?',
   'List the upcoming video schedule for this month']},
 {'name': 'updateVideoSchedule',
  'description': 'Chan

In [ ]:
!pushd {application.generation_dir}/app_schema
!npm install
!npx tsc src/main.ts
!popd


In [ ]:
typespec = application._make_typespec(application_description)
typespec.score, typespec.data.output

In [ ]:
typescript_schema = application._make_typescript_schema(typespec.data.output.typespec_definitions)
typescript_schema.score, typescript_schema.data.output

In [ ]:
drizzle = application._make_drizzle(typespec.data.output.typespec_definitions)
drizzle.score, drizzle.data.output

In [ ]:
router = application._make_router(typespec.data.output.typespec_definitions)
router.score, router.data.output

In [ ]:
handlers = application._make_handlers(
    typespec.data.output.llm_functions,
    typespec.data.output.typespec_definitions,
    typescript_schema.data.output.typescript_schema,
    drizzle.data.output.drizzle_schema,
)

In [ ]:
for name, handler in handlers.items():
    print(name, handler.score, handler.depth, handler.data.output)
    if handler.score != 1:
        print(handler.data.output.feedback["stdout"])

recordExercise 1 0 HandlerOutput(handler='import { db } from "../db";\nimport type { Exercise } from "../common/schema";\nimport { exerciseTable } from "../db/schema/application";\n\ninterface Options {\n    exercise: Exercise;\n}\n\nexport const handle = async (options: Options): Promise<void> => {\n    await db.insert(exerciseTable).values({\n        name: options.exercise.name,\n        sets: options.exercise.sets,\n        reps: options.exercise.reps,\n        weight: options.exercise.weight,\n        notes: options.exercise.notes\n    }).execute();\n};', feedback={'exit_code': 0, 'stdout': None, 'stderr': None})
updateAvailableEquipment 1 0 HandlerOutput(handler='import { db } from "../db";\nimport { equipmentTable } from "../db/schema/application";\nimport type { Equipment } from "../common/schema";\n\ninterface Options {\n    equipment: Equipment[];\n}\n\nexport const handle = async (options: Options): Promise<void> => {\n    // Clear existing equipment\n    await db.delete(equi